### Section 161.1: Setting up py.test

py.test is one of several [third party testing libraries](http://docs.pytest.org/en/latest/) that are available for Python. It can be installed using pip with

In [ ]:
def add(a, b):
    return a + b

In [4]:
%%cmd
py.test

Microsoft Windows [版本 10.0.16299.309]
(c) 2017 Microsoft Corporation。保留所有权利。

E:\MyFile\Jupyter\Python-Learn\Chapter 161 py.test>py.test
============================= test session starts =============================
platform win32 -- Python 3.6.3, pytest-3.2.1, py-1.4.34, pluggy-0.4.0
rootdir: E:\MyFile\Jupyter\Python-Learn\Chapter 161 py.test, inifile:
collected 1 item

test_code.py .

========================== 1 passed in 0.03 seconds ===========================

E:\MyFile\Jupyter\Python-Learn\Chapter 161 py.test>

### Section 161.2: Intro to Test Fixtures

In [ ]:
class Stuff(object):
    
    def prep(self):
        self.foo = 1
        self.bar = 2

In [ ]:
import pytest
from module import stuff
    def test_foo_updates():
        my_stuff = stuff.Stuff()
        my_stuff.prep()
        assert 1 == my_stuff.foo
        my_stuff.foo = 30000
        assert my_stuff.foo == 30000
    def test_bar_updates():
        my_stuff = stuff.Stuff()
        my_stuff.prep()
        assert 2 == my_stuff.bar
        my_stuff.bar = 42
        assert 42 == my_stuff.bar

In [ ]:
import pytest
from module import stuff
    def get_prepped_stuff():
        my_stuff = stuff.Stuff()
        my_stuff.prep()
        return my_stuff
    def test_foo_updates():
        my_stuff = get_prepped_stuff()
        assert 1 == my_stuff.foo
        my_stuff.foo = 30000
        assert my_stuff.foo == 30000
    def test_bar_updates():
        my_stuff = get_prepped_stuff()
        assert 2 == my_stuff.bar
        my_stuff.bar = 42
        assert 42 == my_stuff.bar

**py.test fixtures to the rescue!**

In [ ]:
@pytest.fixture
def prepped_stuff():
    my_stuff = stuff.Stuff()
    my_stuff.prep()
    return my_stuff

In [ ]:
def test_foo_updates(prepped_stuff):
    my_stuff = prepped_stuff
    assert 1 == my_stuff.foo
    my_stuff.foo = 30000
    assert my_stuff.foo == 30000
def test_bar_updates(prepped_stuff):
    my_stuff = prepped_stuff
    assert 2 == my_stuff.bar
    my_stuff.bar = 42
    assert 42 == my_stuff.bar

In [ ]:
def test_foo_updates(prepped_stuff):
    assert 1 == prepped_stuff.foo
    prepped_stuff.foo = 30000
    assert prepped_stuff.foo == 30000
def test_bar_updates(prepped_stuff):
    assert 2 == prepped_stuff.bar
    prepped_stuff.bar = 42
    assert 42 == prepped_stuff.bar

**Cleaning up after the tests are done.**

In [ ]:
@pytest.fixture
def prepped_stuff(request): # we need to pass in the request to use finalizers
    my_stuff = stuff.Stuff()
    my_stuff.prep()
    def fin(): # finalizer function
        # do all the cleanup here
        my_stuff.finish()
    request.addfinalizer(fin) # register fin() as a finalizer
    # you can do more setup here if you really want to
    return my_stuff

In [ ]:
@pytest.yield_fixture
def prepped_stuff(): # it doesn't need request now!
    # do setup
    my_stuff = stuff.Stuff()
    my_stuff.prep()
    # setup is done, pass control to the test functions
    yield my_stuff
    # do cleanup
    my_stuff.finish()

In [ ]:
And that concludes the Intro to Test Fixtures!

For more information, see the [official py.test fixture documentation](http://doc.pytest.org/en/latest/fixture.html) and the [official yield fixture documentation](http://doc.pytest.org/en/latest/yieldfixture.html)

### Section 161.3: Failing Tests

In [ ]:
from module import code
    def test_add__failing():
        assert code.add(10, 11) == 33

In [8]:
%%cmd
py.test

Microsoft Windows [版本 10.0.16299.309]
(c) 2017 Microsoft Corporation。保留所有权利。

E:\MyFile\Jupyter\Python-Learn\Chapter 161 py.test>py.test
============================= test session starts =============================
platform win32 -- Python 3.6.3, pytest-3.2.1, py-1.4.34, pluggy-0.4.0
rootdir: E:\MyFile\Jupyter\Python-Learn\Chapter 161 py.test, inifile:
collected 4 items

test_code.py .
test_fail.py F
test_stuff.py ..

================================== FAILURES ===================================
______________________________ test_add__failing ______________________________

    def test_add__failing():
>       assert my_code.add(10, 11) == 33
E       assert 21 == 33
E        +  where 21 = <function add at 0x0000023E87886950>(10, 11)
E        +    where <function add at 0x0000023E87886950> = my_code.add

test_fail.py:3: AssertionError
===================== 1 failed, 3 passed in 0.09 seconds ======================

E:\MyFile\Jupyter\Python-Learn\Chapter 161 py.test>